# Extract Phase Scope and Channel Correlation Feature from a list of MP3 files

In [1]:
import os
import numpy as np
import librosa
import csv

# Set File Path

In [5]:
path = r'C:\path' #file path that contains mp3 files to be analyzed
mp3list = []
for root, dirs, files in os.walk(path):#loop to list all file paths
    for file in files:
        mp3list.append(os.path.join(root,file))
        
fs = 2048 # frame size
ss = fs # step size

# Define Goniometer Feature Extractor

In [7]:
# Extraction  of all recording studio features
def goniometer(cut, fs, ss):
    phasescope = []
    corr=[]
    for frame in range(int(len(cut[0])/fs)):
        box = [[0,0] for i in range(fs)]
        left = [0 for i in range(fs)]
        for i in range(fs):
            left[i]=cut[0][frame*ss+i]
            box[i]=[int(round(left[i]*10)),int(round(cut[1][frame*ss+i]*10))]
        corr.append(np.min(np.corrcoef(cut[0][frame*ss:frame*ss+fs],cut[1][frame*ss:frame*ss+fs])))
        phasescope.append(len(np.unique(box))/400)
    return phasescope, corr

# Extract from Audio files and write into CSV files 

In [2]:
for n in range(len(mp3list)):
    while True:
        try:
            array, sr=librosa.load(mp3list[n],sr=22050,mono=False)#will resample to 22050 Hz but NOT convert to mono
            print(mp3list[n])
            cut = [array[0][math.floor(len(array[0])/2)-fs*250:math.floor(len(array[0])/2)+fs*250],array[1][math.floor(len(array[0])/2)-fs*250:math.floor(len(array[0])/2)+fs*250]]
# "cut" copies around 45 seconds from the middle of an audio file to extract the goniometer features
# feel free to redefine "cut", e.g. as a complete stereo song or just a bunch of time frames
            phasescope, corr = goniometer(cut, fs, ss)
            features=[np.transpose(phasescope), np.transpose(corr)] 
            Details = ['PhaseScope', 'ChannelCorrelation']  
            with open(mp3list[n]+'.csv', 'w', newline='') as f: 
                write = csv.writer(f) 
                write.writerow(Details) 
                write.writerows(np.transpose(features))
        except:
            break 